# Cybercomp Example - Neuroscience

Here, multiplicity (e.g., Grid Search) should be explicitly defined

## Set Up Code Completions

In [1]:
from pathlib import Path

from cybercomp import API

# generate statically typed objects to define experiments
api = API(server_url="http://127.0.0.1:8765", base_path=Path("./completions"))
api.sync()

Connected to cybercomp server on http://127.0.0.1:8765
[Type] Loaded dl_dataset
[Type] Loaded dl_dataset_name
[Type] Loaded dl_epochs
[Type] Loaded dl_eval_metric
[Type] Loaded dl_model
[Type] Loaded dl_model_name
[Type] Loaded dl_model_weights
[Type] Loaded dl_optimizer
[Type] Loaded dl_probe_layer
[Type] Loaded dl_scheduler
[Type] Loaded neuro_3D_distance
[Type] Loaded neuro_3D_subnet
[Type] Loaded neuro_code
[Type] Loaded neuro_connection_info
[Type] Loaded neuro_current_params
[Type] Loaded neuro_current_stim
[Type] Loaded neuro_cx_cx_g_ampa0
[Type] Loaded neuro_dat
[Type] Loaded neuro_field_file_0
[Type] Loaded neuro_field_file_1
[Type] Loaded neuro_field_file_2
[Type] Loaded neuro_field_file_3
[Type] Loaded neuro_field_file_4
[Type] Loaded neuro_field_file_5
[Type] Loaded neuro_field_file_6
[Type] Loaded neuro_field_file_7
[Type] Loaded neuro_graf_cx
[Type] Loaded neuro_graf_cx6
[Type] Loaded neuro_graf_cxa
[Type] Loaded neuro_graf_in
[Type] Loaded neuro_graf_in6
[Type] Loaded ne

## Import Cybercomp APIs

In [3]:
from cybercomp.functional import experiment, hyperparameter, parameter
from cybercomp.functional import prepare, run, poll, fetch, wait_for_completion

from completions import engines, models
from completions.types import *

## Exploring Available Models

In [4]:
models.neuro_create_source.describe();
models.neuro_create_network.describe();
models.neuro_stim_current.describe();

->[Model] neuro_create_source
  required parameters (2): ['source_defaults', 'source_id']
  optional Parameters (1): ['source_overrides']
  observables (1): ['neuro_code']

->[Model] neuro_create_network
  required parameters (2): ['neuro_code', 'neuro_network_config']
  optional Parameters (3): ['neuro_3D_distance', 'neuro_3D_subnet', 'neuro_mri_network']
  observables (1): ['neuro_connection_info']

->[Model] neuro_stim_current
  required parameters (3): ['neuro_code', 'neuro_connection_info', 'neuro_current_params']
  optional Parameters (0): []
  observables (33): ['neuro_current_stim', 'neuro_cx_cx_g_ampa0', 'neuro_dat', 'neuro_field_file_0', 'neuro_field_file_1', 'neuro_field_file_2', 'neuro_field_file_3', 'neuro_field_file_4', 'neuro_field_file_5', 'neuro_field_file_6', 'neuro_field_file_7', 'neuro_graf_cx', 'neuro_graf_cx6', 'neuro_graf_cxa', 'neuro_graf_in', 'neuro_graf_in6', 'neuro_graf_ina', 'neuro_graf_re', 'neuro_graf_tc', 'neuro_graf_tca', 'neuro_time_G_AMPA0_CX_CX', 'neu

In [ ]:
# @giri: create_model example
# M = create_model(‘g++ -O2 generate_network.cpp -o generete _network’, params_file = ‘networks.cfg’

# output - print out all the parameter (both types and default values)

In [ ]:
# create_model - 
# M = create_model(‘g++ -O2 generate_network.cpp -o generete _network’, params_file = ‘networks.cfg’

# output - print out all the parameter (both types and default values)

## Defining a Multi-Step Experiment

Here, you can define individual experiments (i.e., steps) and chain them into larger experiments, if need be.
Internally, Cybercomp will connect the outputs from previous experiments into subsequent experiments that depend on them.
Users only need to provide the non-inferrable parameters / hyperparameters.

In [6]:
# preprocess = experiment(models.sub_source(), engines.mustache_engine(), name="sub_source")
# hide the top part as internal implementation

# print out the description of the experiment

# create_model - 
# M = create_model(‘g++ -O2 generate_network.cpp -o generete _network’, params_file = ‘networks.cfg’
small_experiment_0 = experiment(models.neuro_create_source(), engines.mustache(), name="make_source")
small_experiment_1 = experiment(models.neuro_create_network(), engines.neuro_sleep_stage(), name="make_network")
small_experiment_2 = experiment(models.neuro_stim_current(), engines.neuro_sleep_stage(), name="simulate_network")

# large_experiment = (preprocess >> small_experiment_1 >> small_experiment_2)
large_experiment = (small_experiment_0 >> small_experiment_1 >> small_experiment_2)
large_experiment.describe()

->[Experiment] make_source+make_network+simulate_network

  ->[Experiment] make_source+make_network

    ->[Experiment] make_source

      ->[Step] make_source
        model=neuro_create_source
        engine=mustache

        ->[Model] neuro_create_source
          required parameters (2): ['source_defaults', 'source_id']
          optional Parameters (1): ['source_overrides']
          observables (1): ['neuro_code']

        ->[Engine] mustache
          hyperparameters (1): ['source_id']

    ->[Experiment] make_network

      ->[Step] make_network
        model=neuro_create_network
        engine=neuro_sleep_stage

        ->[Model] neuro_create_network
          required parameters (2): ['neuro_code', 'neuro_network_config']
          optional Parameters (3): ['neuro_3D_distance', 'neuro_3D_subnet', 'neuro_mri_network']
          observables (1): ['neuro_connection_info']

        ->[Engine] neuro_sleep_stage
          hyperparameters (0): []

  ->[Experiment] simulate_network

 

## Running the Experiment

First, let's define the runtime for this experiment. For this example, we will use the local runtime (i.e., the machine this notebook will be executed on).

In [ ]:
# run experiment
from cybercomp.runtimes import LocalRuntime

with LocalRuntime(reuse_past_runs=True) as r:
    ref = run(
        large_experiment,
        (
            parameter(neuro_connection_info, "samples/sleep_stage_transition/connection_info"),
            parameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
        ),
        runtime=r,
    )
    fetch(*ref, runtime=r, observables=[neuro_connection_info])

with LocalRuntime(reuse_past_runs=True) as r:

    def run_experiment(experiment, *params, runtime, observables):

        plan = prepare(large_experiment, *params)

        # execute the large experiment, and get a reference to it
        run_state = run(large_experiment, *plan, runtime=r)

        # wait until large experiment is finished
        wait_for_completion(large_experiment, *plan, runtime=r)

        # fetch an output from large experiment execution
        connection_info = fetch(large_experiment, *plan, runtime=r, observables=[neuro_connection_info])
        return connection_info

    # prepare the large experiment for execution
    connection_info = run_experiment(
        large_experiment,
        (
            parameter(neuro_network_config, "samples/sleep_stage_transition/network.cfg"),
            parameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
        ),
        runtime=r,
        observables=[neuro_connection_info],
    )

    # prepare a new experiment with the fetched output as a parameter
    small_experiment = experiment(models.update_network(), engines.bazhenov_labs_engine(), name="update_network")

    # prepare small experiment from large_experiment observations
    plan = prepare(
        small_experiment,
        (
            parameter(neuro_connection_info, connection_info),
            parameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
        ),
    )

    # execute the small experiment, and get a reference to it
    run_state = run(small_experiment, *plan, runtime=r)

    # wait until small experiment is finished
    wait_for_completion(small_experiment, *plan, runtime=r)

    # fetch an output from small experiment execution
    updated_connection_info = fetch(small_experiment, *plan, runtime=r, observables=[neuro_connection_info])

In [ ]:
# run a grid search on s2, using output from s1
s1 = experiment(models.make_network(), engines.bazhenov_labs_engine(), name="make_network")
s1.describe()

s1.run()

s2 = experiment(models.make_apples(), engines.bazhenov_labs_engine(), name="simulate_network")

s1 = ['x', 'y'] -> ['neuro_connection_info']
s2 = ['neuro_connection_info', 'a'] -> ['d']

s1b = experiment(models.modify_network(), engines.bazhenov_labs_engine(), name="modify_network")
s = (s1 >> s1b)

# 'a' = "val_a"
# 'b' = "val_b"
# s2 = ['a', 'b', 'neuro_connection_info'] -> ['d']

# as a requirement, s2 should take in a 'neuro_connection_info' as an input

s1 >> s2 >> s3

['z', 'd']

exp = (s1 >> s2)

# args
args = (
    parameter(neuro_network_config, "samples/sleep_stage_transition/network.cfg"),
    parameter(neuro_connection_info, "samples/sleep_stage_transition/connection_info"),
    hyperparameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
)

In [ ]:
%run_remote

output = ex1 > ex2 > ex3

In [ ]:
%run_local

output > ex4

In [7]:
range_P = [
    P.Range(models.stim_current.neuro_connection_info, 0.1, 1.0, 10),
    P.Range(models.stim_current.neuro_current_params, 0.1, 1.0, 10),
]
range_H = [
    H.Range(engines.bazhenov_labs_engine.neuro_current_params, 0.1, 1.0, 10),
]

In [ ]:
sweep1 = Experiment.Sweep("bazhenov_network_grid_search", (seq1,), range_P, range_H, []) # change syntax
sweep1.run(runtime)
exChain_obs = sweep1.fetch(runtime)

In [ ]:
# run sweep
for p in range_P:
    for h in range_H:
        

In [ ]:
ex12 = Collection(
    name="simulate_bhazenov_network",
    models=[models.make_network, models.stim_current],
    engine=engines.bazhenov_labs_engine,
    parameters=[
        models.make_network.neuro_network_config("samples/sleep_stage_transition/params.txt"),
        models.stim_current.neuro_current_params("samples/sleep_stage_transition/connection_info")
    ],
    parameters={
        "neuro_network_config": P("samples/sleep_stage_transition/network.cfg"),
        "neuro_current_params": P("samples/sleep_stage_transition/params.txt"),
    },
    hyperparameters={
        "neuro_current_params": H("samples/sleep_stage_transition/params.txt"),
    },
    observables={
        "neuro_time_cx6": O("observables/time_cx6")
    },
)

runtime = Runtime()
ex2.execute(runtime)
ex2_obs = ex2.gather_observables(runtime)

## POC - double pendulum simulation
## Real - replicate simone's experiments


# the intersecting observables are piped into the params of the next experiment
# only the intersecting observables are shown in autocompletion

experiment_2 = create_new_experiment_from(experiment_1)
experiment_2.name = "<>"
experiment_2.parameters["neuro/network_config"] = "new-value"
experiment.check_existing()

experiment.observables

experiemnt.validate()
experiment.run(
hpc_recipe={}
)

# !!! [collection] - a set of experiments with common observables

# observables may be a huge list, so need not provide everytime when its

implictly discoverable

# to get experiments run with different observables

collection = create_collection(
model="name_of_model",
parameters={
"neuro/network_config": [],
},
)

# the collection experiments are pulled from the db

collection = create_collection(
model=["model1", "model2", ...],
parameters={
"neuro/network_config": [],
},
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

# all experiments sharing the same observables

collection = create_collection(
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

collection = [experiment_1, experiment_2]

# experiment collection

# example of experiment chaining (top-to-bottom mro)

# example 1

experiment = Experiment(
experiment_1,
experiment_2,
)

# example 2

experiment = Experiment(
[experiment_2, experiment_3, .....], #
experiment_1, #
)

#

[
exp2 -> exp1,
exp3 -> exp1,
]

# example 3

experiment = Experiment(
[experiment_2, ...collection.experiments],
experiment_1,
)

In [ ]:
# analysis part =========================

# takes a collection as input,

# and runs some function over the observables on that

# collection

# a primitive form of experiment using a collection of experiments as input

analysis = Analysis(
collection=[],
function={

    }

)

analysis = experiment